# Metaprogramming

A "meta" program is a program that manipulates programs.

Most common use refers to a program that *generates* another program.

In the last class we saw how Julia can generate specialized code for you.

Sometimes that's not enough, and you need to write a program to explicitly generate the code needed for a specialized problem.

Julia allows us to talk in a "meta" way ("one level up"), about Julia code,  that is to "treat code as data" and manipulate it as just another object in Julia. (This is very similar to Lisp.)

## Symbols and Expressions

One of the most basic objects in this approach are unevaluated *symbols*:

In [2]:
:a  # "the symbol a"

:a

In [3]:
typeof(:a)

Symbol

`:a` refers to the symbol `a`. We can evaluate it with the `eval` function:

In [4]:
eval(:a)

LoadError: [91mUndefVarError: a not defined[39m

`a` must be defined for this to work:

In [5]:
a = 3

3

In [6]:
:a

:a

In [7]:
eval(:a)

3

The `eval` function takes an expression and evaluates it, that is, *generates the corresponding code*

Operators and function names are also symbols:

In [8]:
:+, :sin

(:+, :sin)

In [9]:
typeof(:+)

Symbol

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [10]:
ex = :(a + b)  # the expression 'a + b'

:(a + b)

In [11]:
typeof(ex)

Expr

In [12]:
b = 7
eval(ex)

10

An expression is just a Julia object, so we can introspect (find out information about it):

In [13]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b
  typ: Any


In [14]:
dump(:(x = 3))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 3
  typ: Any


In [ ]:
# ex.<TAB>

In [17]:
ex.head

:call

In [16]:
ex.args

3-element Array{Any,1}:
 :+
 :a
 :b

The job of Julia's parser is to convert a sequence of characters into these `Expr` objects:

In [18]:
parse("a + b")

:(a + b)

More complicated expressions are represented as "abstract syntax trees" (ASTs), consisting of expressions nested inside expressions:

In [19]:
ex = :( sin(3a + 2b^2) )

:(sin(3a + 2 * b ^ 2))

In [20]:
ex.args

2-element Array{Any,1}:
 :sin             
 :(3a + 2 * b ^ 2)

In [21]:
typeof(ex.args[2])

Expr

In [22]:
ex.args[2].args

3-element Array{Any,1}:
 :+          
 :(3a)       
 :(2 * b ^ 2)

In [23]:
ex.args[1] = :cos

:cos

In [24]:
ex

:(cos(3a + 2 * b ^ 2))

In [25]:
blk = quote
    println("Hello")
end

quote  # In[25], line 2:
    println("Hello")
end

In [26]:
eval(blk)

Hello


In [27]:
dump(blk)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: Expr
      head: Symbol line
      args: Array{Any}((2,))
        1: Int64 2
        2: Symbol In[25]
      typ: Any
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol println
        2: String "Hello"
      typ: Any
  typ: Any


In [28]:
push!(blk.args, :(println("AFTER")))

3-element Array{Any,1}:
 :( # In[25], line 2:)
 :(println("Hello"))  
 :(println("AFTER"))  

In [29]:
blk

quote  # In[25], line 2:
    println("Hello")
    println("AFTER")
end

In [30]:
eval(blk)

Hello
AFTER


In [31]:
unshift!(blk.args, :(println("BEFORE")))

4-element Array{Any,1}:
 :(println("BEFORE")) 
 :( # In[25], line 2:)
 :(println("Hello"))  
 :(println("AFTER"))  

In [32]:
eval(blk)

BEFORE
Hello
AFTER


Expressions can be arbitrary Julia code that when evaluated will have side effects. For longer blocks of code, `quote...end` may be used instead of `:( ... )`

In [33]:
ex2 = 
quote
    y = 3
    z = sin(y+1)
end

quote  # In[33], line 3:
    y = 3 # In[33], line 4:
    z = sin(y + 1)
end

In [34]:
eval(ex2)
y

3

In [35]:
eval(ex2)
z

-0.7568024953079282

The full form of the abstract syntax tree in a style similar to a Lisp s-expression can be obtained using functions from the `Meta` module in `Base`:

In [36]:
Meta.show_sexpr(ex2)

(:block,
  (:line, 3, Symbol("In[33]")),
  (:(=), :y, 3),
  (:line, 4, Symbol("In[33]")),
  (:(=), :z, (:call, :sin, (:call, :+, :y, 1)))
)

Another way of seeing the structure is with `dump`:

In [ ]:
dump(ex2)

## Interpolation

Values can be *interpolated* into expressions with `$`.

In [37]:
x = 2

2

In [38]:
:(x + $x)

:(x + 2)

In [41]:
x = :(y + z)

:(y + z)

In [43]:
:(x + $(1+3*4))

:(x + 13)

In [45]:
Expr(:call, :+, :x, 1 + 3*4)

:(x + 13)

### Why not strings?

*Abstract* syntax vs. *concrete* syntax.

In [46]:
a = "x + y"

"x + y"

In [47]:
b = "z"

"z"

In [49]:
expr = "($a) * $b"

"(x + y) * z"

Even worse, what if `b == ")"`?

With `Expr` objects, every expression is well-formed and each slot has its abstract meaning.

### Splat interpolation

In [51]:
vars = [:a, :b, :c]

3-element Array{Symbol,1}:
 :a
 :b
 :c

In [55]:
:(1 - f(0, $(vars...)))

:(1 - f(0, a, b, c))

### `@eval` for interpolating on the fly

You will sometimes see `@eval expr`, which is shorthand for:

```
eval(quote
         expr
     end)
```

allowing items to be interpolated immediately before executing.

In [56]:
fnames = [ Symbol("func$i") for i=1:10]

10-element Array{Symbol,1}:
 :func1 
 :func2 
 :func3 
 :func4 
 :func5 
 :func6 
 :func7 
 :func8 
 :func9 
 :func10

In [57]:
for i = 1:length(fnames)
    @eval ($(fnames[i]))(x) = $i
end

In [58]:
func1(0)

1

In [60]:
func9(0)

9

## Macros

With the ability to think of code in terms of a data structure in the Julia language, we can now *manipulate* those data structures, allowing us to *create Julia code from within Julia*.

*Macros* provide a particular use pattern of metaprogramming: replacing one expression with another, in-place, right after parsing.

The [Julia manual](http://julia.readthedocs.org/en/latest/manual/metaprogramming/) puts it like this: 

> macros map a tuple of argument *expressions* to a returned
> *expression*

Macros are useful in several cases:

- to provide a specific notation different than what can normally be written in the language
    - e.g. https://github.com/JuliaOpt/JuMP.jl/blob/release-0.18/examples/sudoku.jl#L44
- to rearrange or delay evaluation of code
- to eliminate boilerplate (repetitive) code
- to automatically generate complex code that would be painful by hand
- to unroll loops for efficiency
- to inline code for efficiency



Macros are invoked using the `@` sign, e.g.

In [ ]:
time(()->sin(10))

In [62]:
@time sin(10)

  0.000005 seconds (5 allocations: 176 bytes)


-0.5440211108893698

A trivial example of defining a macro is the following, which runs whatever code it is passed two times.

In [1]:
macro twice(ex)
    quote
        $ex
        $ex
    end
end

@twice (macro with 1 method)

In [4]:
x = 0
@twice println("hello")

hello
hello


We can see what effect the macro actually has using `macroexpand`:

In [6]:
macroexpand(:(@twice println("hello")))

quote  # In[1], line 3:
    (Main.println)("hello") # In[1], line 4:
    (Main.println)("hello")
end

In [7]:
macroexpand(:(@time sin(10)))

quote  # util.jl, line 235:
    local #1#stats = (Base.gc_num)() # util.jl, line 236:
    local #3#elapsedtime = (Base.time_ns)() # util.jl, line 237:
    local #2#val = sin(10) # util.jl, line 238:
    #3#elapsedtime = (Base.time_ns)() - #3#elapsedtime # util.jl, line 239:
    local #4#diff = (Base.GC_Diff)((Base.gc_num)(), #1#stats) # util.jl, line 240:
    (Base.time_print)(#3#elapsedtime, #4#diff.allocd, #4#diff.total_time, (Base.gc_alloc_count)(#4#diff)) # util.jl, line 242:
    #2#val
end

In [8]:
macro mytime(ex)
    quote
        t0 = time()
        val = $ex
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

@mytime (macro with 1 method)

In [22]:
@macroexpand @mytime (sleep(1); "done")

quote  # In[8], line 3:
    #13#t0 = (Main.time)() # In[8], line 4:
    #14#val = begin 
            (Main.sleep)(1)
            "done"
        end # In[8], line 5:
    #15#t1 = (Main.time)() # In[8], line 6:
    (Main.println)("$(#15#t1 - #13#t0) seconds elapsed") # In[8], line 7:
    #14#val
end

A line break terminates macro arguments, so blocks need to be passed with `begin...end`:

In [ ]:
@time begin
    # code
end

----
**Exercise**: Define a macro `@until` that does an `until` loop.

----

In [13]:
macro until(condition, expression)
    quote
        $(esc(expression))
        while !$(esc(condition))
            $(esc(expression))
        end
    end
end

@until (macro with 1 method)

In [16]:
x = [10]
@until x[1] < 0 begin
    @until true begin
        x[1] -= 1
        println(x[1])
    end
end

9
8
7
6
5
4
3
2
1
0
-1


### Macro Hygiene

In [27]:
macro set_x(val)
    :(x = $val)
end

@set_x (macro with 1 method)

In [28]:
@macroexpand @set_x 10

:(#16#x = 10)

In [21]:
x

1-element Array{Int64,1}:
 -1

In [29]:
macro set_x(val)
    :($(esc(:x)) = $val)
end

@set_x (macro with 1 method)

In [30]:
@macroexpand @set_x 10

:(x = 10)

In [26]:
x

10

### Macros for numerical performance: Horner's method

There are many interesting examples of macros in `Base`. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$

may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a `for` loop. But if we know the polynomial *at compile time*, this loop may be unrolled using metaprogramming. This is implemented in the `Math` module in `math.jl` in `Base`, so the name of the macro (which is not exported) is `@Base.Math.horner`.

In [31]:
# copied from base/math.jl
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )  # actually uses `muladd`
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

This is called as follows: to evaluate the polynomial $p(x) = 2 + 3x + 4x^2$ at $x=3$, we do

In [34]:
x = 3
@horner(x, 2, 3, 4, 5)

182

To see what the macro does to this call, we again use `macroexpand`:

In [33]:
macroexpand(:(@horner(sin(x-y), 2, 3, 4, 5, 6, 7, 8, 9, 10)))

quote 
    #19#t = sin(x - y)
    2 + #19#t * (3 + #19#t * (4 + #19#t * (5 + #19#t * (6 + #19#t * (7 + #19#t * (8 + #19#t * (9 + #19#t * 10)))))))
end

In [35]:
macroexpand(:(@Base.Math.horner(x, 2, 3, 4, 5)))

quote 
    #21#t = x
    (Base.Math.muladd)(#21#t, (Base.Math.muladd)(#21#t, (Base.Math.muladd)(#21#t, 5, 4), 3), 2)
end

In [36]:
?muladd

search: muladd



```
muladd(x, y, z)
```

Combined multiply-add, computes `x*y+z` in an efficient manner. This may on some systems be equivalent to `x*y+z`, or to `fma(x,y,z)`. `muladd` is used to improve performance. See [`fma`](@ref).

# Example

```jldoctest
julia> muladd(3, 2, 1)
7

julia> 3 * 2 + 1
7
```


In [37]:
?fma

search: fma findmax findmax! DateFormat @fastmath @dateformat_str UniformScaling



```
fma(x, y, z)
```

Computes `x*y+z` without rounding the intermediate result `x*y`. On some systems this is significantly more expensive than `x*y+z`. `fma` is used to improve accuracy in certain algorithms. See [`muladd`](@ref).


In [42]:
f(x) = Base.Math.@evalpoly(x, 1.2, 2.3, 3.4, 4.5)

f (generic function with 1 method)

In [44]:
@code_llvm f(0.1 + 0.2im)


define void @julia_f_61270(%Complex.62* noalias nocapture sret, %Complex.62* nocapture readonly dereferenceable(16)) #0 !dbg !5 {
top:
  %2 = getelementptr inbounds %Complex.62, %Complex.62* %1, i64 0, i32 0
  %3 = load double, double* %2, align 8
  %4 = getelementptr inbounds %Complex.62, %Complex.62* %1, i64 0, i32 1
  %5 = load double, double* %4, align 8
  %6 = fadd double %3, %3
  %7 = fmul double %5, %5
  %8 = call double @llvm.fmuladd.f64(double %3, double %3, double %7)
  %9 = call double @llvm.fmuladd.f64(double %6, double 4.500000e+00, double 3.400000e+00)
  %10 = fmul double %8, 4.500000e+00
  %11 = fsub double 2.300000e+00, %10
  %12 = call double @llvm.fmuladd.f64(double %6, double %9, double %11)
  %13 = fmul double %9, %8
  %14 = fsub double 1.200000e+00, %13
  %15 = call double @llvm.fmuladd.f64(double %3, double %12, double %14)
  %16 = fmul double %5, %12
  %.sroa.0.0..sroa_idx = getelementptr inbounds %Complex.62, %Complex.62* %0, i64 0, i32 0
  store double %15, do

In [46]:
@code_native f(0.1 + 0.2im)

	.text
Filename: In[42]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 129
	vmovsd	(%rsi), %xmm0           # xmm0 = mem[0],zero
	vmovsd	8(%rsi), %xmm1          # xmm1 = mem[0],zero
	vaddsd	%xmm0, %xmm0, %xmm2
	vmulsd	%xmm1, %xmm1, %xmm3
	vmulsd	%xmm0, %xmm0, %xmm4
	vaddsd	%xmm3, %xmm4, %xmm3
	movabsq	$140259702279240, %rax  # imm = 0x7F90C1BB0C48
	vmovsd	(%rax), %xmm4           # xmm4 = mem[0],zero
	vmulsd	%xmm4, %xmm2, %xmm5
	movabsq	$140259702279248, %rax  # imm = 0x7F90C1BB0C50
	vaddsd	(%rax), %xmm5, %xmm5
	vmulsd	%xmm4, %xmm3, %xmm4
	movabsq	$140259702279256, %rax  # imm = 0x7F90C1BB0C58
	vmovsd	(%rax), %xmm6           # xmm6 = mem[0],zero
	vsubsd	%xmm4, %xmm6, %xmm4
	vmulsd	%xmm5, %xmm2, %xmm2
	vaddsd	%xmm4, %xmm2, %xmm2
	vmulsd	%xmm3, %xmm5, %xmm3
	movabsq	$140259702279264, %rax  # imm = 0x7F90C1BB0C60
	vmovsd	(%rax), %xmm4           # xmm4 = mem[0],zero
	vsubsd	%xmm3, %xmm4, %xmm3
	vmulsd	%xmm2, %xmm0, %xmm0
	vaddsd	%xmm3, %xmm0, %xmm0
	vmulsd	%xmm2, %xmm1, %xmm1
	vmovsd	%xmm0, (%rdi

## `@generated` Functions

Another form of metaprogramming available in Julia.

Generate code based on *types* instead of the input expressions.

In [49]:
@generated function nloops(a::Array{T,N}) where {T,N}
    # inside here,
    # - `a` refers to the type of the argument
    # - `T` and `N` have their usual values
    # - return an expression to compute the answer for this type
    vars = [Symbol("i$i") for i in 1:N]
    ex = :(println(join([$(vars...)]," ")))
    for i in 1:N
        ex = quote
            for $(vars[i]) in 1:size(a,$i)
                $ex
            end
        end
    end
    return ex
end

nloops (generic function with 1 method)

In [51]:
nloops(rand(2,3,4))

1 1 1
2 1 1
1 2 1
2 2 1
1 3 1
2 3 1
1 1 2
2 1 2
1 2 2
2 2 2
1 3 2
2 3 2
1 1 3
2 1 3
1 2 3
2 2 3
1 3 3
2 3 3
1 1 4
2 1 4
1 2 4
2 2 4
1 3 4
2 3 4


In [52]:
parse("r\"a*\"")

:(@r_str "a*")

In [53]:
using PyCall

INFO: Precompiling module PyCall.


In [54]:
x = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [55]:
py"sum($x)"

6

In [58]:
p = py"lambda x: x*x"

PyObject <function <lambda> at 0x7f90a8ea5668>

In [59]:
p(4)

16

In [60]:
g = x->x*x

(::#5) (generic function with 1 method)

In [61]:
g(4)

16

In [62]:
@macroexpand py"lambda x: x*x"

quote  # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 214:
    #51#m = (PyCall.maindict)() # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 215:
    begin 
    end # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 216:
    #52#ret = (PyCall.PyAny)((PyCall.pyeval_)(Base.string("lambda x: x*x"), #51#m, #51#m, 258, "/home/jeff/.julia/v0.6/PyCall/src/pyeval.jl")) # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 217:
    begin 
    end # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 218:
    #52#ret
end

In [66]:
@macroexpand py"1+2"o

quote  # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 214:
    #59#m = (PyCall.maindict)() # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 215:
    begin 
    end # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 216:
    #60#ret = (PyCall.PyObject)((PyCall.pyeval_)(Base.string("1+2"), #59#m, #59#m, 258, "/home/jeff/.julia/v0.6/PyCall/src/pyeval.jl")) # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 217:
    begin 
    end # /home/jeff/.julia/v0.6/PyCall/src/pyeval.jl, line 218:
    #60#ret
end

In [1]:
macro foo_str(s, options...)
    println(s)
    println(options)
    0
end

@foo_str (macro with 1 method)

In [3]:
foo"a/b/c"fhdsj

a/b/c
("fhdsj",)


0